In [1]:
import pandas as pd
import numpy as np

In [6]:
#load in datasets
fake = pd.read_csv('data\Fake.csv')
real = pd.read_csv('data\True.csv')

In [7]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


Since the text of true.csv has a certain format ('STATE (Reuters) - text'), this beginning clause needs to be removed to prevent data leakage <br>
Note: some rows are missing 'STATE', some are missing '(Reuters)', and some are missing both

In [9]:
#Create column that is True if text starts with an upper case word
def uppercase(text):
    words = text.split()
    if len(words) >0:
        return words[0].isupper()
    else:
        return False
    
real['uppercase'] =  real.text.apply(uppercase)

In [10]:
#Create column that is True if the text contains the string '(Reuters)'
real['reuters'] = real.text.str.contains('\(Reuters\)')

In [21]:
#check out special cases with uppercase first word but no '(Reuters)'
special_cases = real[(real['uppercase'] == True) & (real['reuters'] == False)]
print(len(special_cases))
special_cases.head(10)

52


,title,text,subject,date,uppercase,reuters
103,Democratic U.S. senator seeks audit of EPA chi...,WASHINGTON () - The top Democrat on the Senate...,politicsNews,"December 18, 2017",True,False
231,Special Counsel Mueller filing shows Manafort ...,WASHINGTON (Reuters on Friday ) - U.S. Special...,politicsNews,"December 8, 2017",True,False
272,Exclusive: U.S. regulators offer Congress oliv...,NEW YORK (LPC/IFR) - U.S. regulators said they...,politicsNews,"December 7, 2017",True,False
427,Factbox: Republicans to watch in U.S. Senate t...,WASHINGTON - Some key U.S. senators still had ...,politicsNews,"November 30, 2017",True,False
1141,GAO opens door for Congress to review leverage...,NEW YORK (IFR) - The investigative arm of Cong...,politicsNews,"October 19, 2017",True,False
4465,Window closing for Republican stealth assault ...,WASHINGTON The clock began running out this we...,politicsNews,"April 4, 2017",True,False
5363,Trump's defense chief visits UAE in first Midd...,ABU DHABI - U.S. President Donald Trump’s defe...,politicsNews,"February 18, 2017",True,False
5821,Kushner divests equity in major NYC property,NEW YORK (IFR) - Jared Kushner has divested hi...,politicsNews,"January 31, 2017",True,False
6089,Trump makes early move on restricting abortion...,NEW YORK (Thomson Reuters Foundation) - U.S. P...,politicsNews,"January 23, 2017",True,False
6313,Davos elites struggle for answers as Trump era...,"DAVOS, Switzerland - The global economy is in ...",politicsNews,"January 15, 2017",True,False


cases: <br>
reuters false AND uppercase false - do not split <br>
reuters true OR uppercase true - split <br>

\**special cases: 4465, 8247, 9618 - do not split\** (first word is uppercase but not followed by a '-')

In [12]:
real.iloc[[4465,8247,9618]]

,title,text,subject,date,uppercase,reuters
4465,Window closing for Republican stealth assault ...,WASHINGTON The clock began running out this we...,politicsNews,"April 4, 2017",True,False
8247,Obama drinks coconut water to cool down in Laos,U.S. President Barack Obama visited a street m...,politicsNews,"September 7, 2016",True,False
9618,Commentary: Party leaders often disliked their...,GOP leaders have unleashed a stunning level of...,politicsNews,"May 10, 2016",True,False


In [8]:
#Create column that is true if it can be split on '- '
real['split'] = (real.uppercase == True) | (real.reuters == True)

#exceptions
for i in [4465, 8247, 9618]:
    real.loc[i, 'split'] = False

In [9]:
#strip beginning of rows in real with 'STATE (Reuters) - ' format
def strip_beginning(row, split, text):
    if row[split]:
        return row[text].partition('- ')[2]
    else:
        return row[text]
    
#overwrite text column with beginning stripped
real['text'] = real.apply(lambda x: strip_beginning(x,'split', 'text'), axis=1)

In [10]:
#drop dummy variables
real.drop(['uppercase', 'reuters', 'split'], axis=1, inplace=True)

In [11]:
#create label column: 1 if fake, 0 if True
fake['label']= 1
real['label'] = 0

In [12]:
#combine datasets
df = pd.concat([fake, real], ignore_index=True)

In [13]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [15]:
#drop duplicates with the same title and text
dup = df[df.duplicated(['title', 'text'])]
dup

,title,text,subject,date,label
9942,HILLARY TWEETS MESSAGE In Defense Of DACA…OOPS...,No time to waste we've got to fight with eve...,politics,"Sep 9, 2017",1
11446,FORMER DEMOCRAT WARNS Young Americans: “Rioter...,"Who is silencing political speech, physically...",politics,"Mar 10, 2017",1
14925,[VIDEO] #BlackLivesMatter Terrorists Storm Dar...,They were probably just looking for a safe sp...,politics,"Nov 16, 2015",1
15892,HOUSE INTEL Slaps Subpoenas on McCain Institut...,Please see our previous report below on the Mc...,Government News,"Dec 27, 2017",1
15893,PRICELESS! WATCH MSNBC HOST’S Shocked Response...,THIS IS PRICELESS! The video below shows just ...,Government News,"Dec 26, 2017",1
...,...,...,...,...,...
44709,France unveils labor reforms in first step to ...,French President Emmanuel Macron s government ...,worldnews,"August 31, 2017",0
44744,Guatemala top court sides with U.N. graft unit...,Guatemala s top court on Tuesday ruled definit...,worldnews,"August 29, 2017",0
44771,"Europeans, Africans agree renewed push to tack...",Europe s big four continental powers and thr...,worldnews,"August 28, 2017",0
44834,Thailand's ousted PM Yingluck has fled abroad:...,Ousted Thai prime minister Yingluck Shinawatra...,worldnews,"August 25, 2017",0


In [16]:
df.drop(dup.index, inplace=True)

In [17]:
df.reset_index(drop=True, inplace=True)

In [18]:
df

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
39100,'Fully committed' NATO backs new U.S. approach...,NATO allies on Tuesday welcomed President Dona...,worldnews,"August 22, 2017",0
39101,LexisNexis withdrew two products from Chinese ...,"LexisNexis, a provider of legal, regulatory an...",worldnews,"August 22, 2017",0
39102,Minsk cultural hub becomes haven from authorities,In the shadow of disused Soviet-era factories ...,worldnews,"August 22, 2017",0
39103,Vatican upbeat on possibility of Pope Francis ...,Vatican Secretary of State Cardinal Pietro Par...,worldnews,"August 22, 2017",0


In [21]:
#export combined data
df.to_csv('data\combined.csv')